<a href="https://colab.research.google.com/github/shw9807/shw9807blog/blob/master/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C_%EC%97%B0%EC%B2%B4%EC%98%88%EC%B8%A1_%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "[Colab] 신용카드 사용자 연체 예측 - 하이퍼 파라미터"

- toc:true
- branch: master
- badges: true
- author: shw9807
- categories: [데이콘]

- 하이퍼파라미터 튜닝에 쓰고 있는 최신 Automl 기법이다.
- 빠르게 튜닝이 가능하다는 장점이 있다.
- 하이퍼파라미터 튜닝 방식을 지정할수 있다. -> 직관적인 api인 튜닝된 lightgbm도 제공해준다.
- 다른 라이브러리들에 비해 직관적인 장점이 있어 코딩하기 용이하다.

In [ ]:
!pip install optuna

     |████████████████████████████████| 308 kB 8.9 MB/s 
     |████████████████████████████████| 210 kB 49.2 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 113 kB 49.6 MB/s 
     |████████████████████████████████| 149 kB 47.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=29461bb9e67280164fa0d16e167bbc12687ae02b465ef8870baaa064a101a4df
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMClassifier
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

## 간단한 전처리

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/신용카드 연체예측/"
train = pd.read_csv(path + "train.csv")
train = train.drop(["index"], axis=1)
train.fillna("NAN", inplace=True)

test = pd.read_csv(path + "test.csv")
test = test.drop(["index"], axis=1)
test.fillna("NAN", inplace=True)

In [ ]:
train_ohe = pd.get_dummies(train)
test_ohe = pd.get_dummies(test)

In [ ]:
X = train_ohe.drop(["credit"], axis=1)
y = train["credit"]
X_test = test_ohe.copy()

- Optuna는 objective하이퍼 파라미터의 성능을 평가하고 향후 시험에서 샘플링 할 위치를 결정하기 위해 숫자 값을 반환 하는 함수가 필요하다는 것을 의미하는 블랙 박스 최적화 프로그램 이다.
- Optuna의 특정 인수를 object에 전달된다.
- trial는 조정해야하는 하이퍼 파라미터를 지정 하기 위해 objective 함수에 전달된다.
- 이것은 logloss 성능에 대한 피드백으로 Optuna에서 사용하는 모델에서 반환한다.

In [ ]:
def objective(trial: Trial) -> float:
    params_lgb = {
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "objective": "multiclass",
        "metric": "multi_logloss",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

    model = LGBMClassifier(**params_lgb)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    lgb_pred = model.predict_proba(X_valid)
    log_score = log_loss(y_valid, lgb_pred)
    
    return log_score

In [ ]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2022-02-10 08:39:17,315] A new study created in memory with name: lgbm_parameter_opt
[I 2022-02-10 08:39:40,151] Trial 0 finished with value: 0.7390804911846213 and parameters: {'reg_alpha': 1.12424581642324e-05, 'reg_lambda': 0.08556428806974939, 'max_depth': 15, 'num_leaves': 154, 'colsample_bytree': 0.4936111842654619, 'subsample': 0.40919616423534183, 'subsample_freq': 1, 'min_child_samples': 88, 'max_bin': 380}. Best is trial 0 with value: 0.7390804911846213.
[I 2022-02-10 08:39:52,191] Trial 1 finished with value: 0.7319514638826676 and parameters: {'reg_alpha': 2.1245096608103405e-05, 'reg_lambda': 0.0018526142807772773, 'max_depth': 20, 'num_leaves': 214, 'colsample_bytree': 0.5274034664069657, 'subsample': 0.42727747704497043, 'subsample_freq': 2, 'min_child_samples': 34, 'max_bin': 357}. Best is trial 1 with value: 0.7319514638826676.
[I 2022-02-10 08:40:08,554] Trial 2 finished with value: 0.7463491007324939 and parameters: {'reg_alpha': 1.2964031109077052e-05, 'reg_lambd

Best Score: 0.7234749185219674
Best trial: {'reg_alpha': 1.987904330777592e-05, 'reg_lambda': 0.028054003730936226, 'max_depth': 11, 'num_leaves': 141, 'colsample_bytree': 0.5109126733153162, 'subsample': 0.9787092394351908, 'subsample_freq': 8, 'min_child_samples': 95, 'max_bin': 469}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 각 파라미터들의 상관관계
optuna.visualization.plot_contour(
    study,
    params=[
        "max_depth",
        "num_leaves",
        "colsample_bytree",
        "subsample",
        "subsample_freq",
        "min_child_samples",
        "max_bin",
    ],
)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

하이퍼파라미터 중요도는 전체 파라미터들의 영향력의 총합을 1이라고 생각했을 때  
해당 하이퍼파라미터가 어느정도의 영향력을 가지고 있는지를 나타내준다.  
min_child_samples의 경우에는 거의 0에 가까운 영향력을 가지고 있지만  
과적합을 피하기 위해서는 사용하는 것이 좋다.